In [68]:
import cv2
import numpy as np
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint 
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from scipy.misc import imsave

In [74]:
img_width, img_height = 32,32

train_data_dir = '../SmallTelugu/train'
validation_data_dir = '../SmallTelugu/test/'
nb_train_samples = 68400
nb_validation_samples = 20520
epochs = 15
batch_size = 32
CLASSES=[str(i) for i in range(1,685)]

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [75]:
model = Sequential()
    
model.add(Conv2D(20, (3, 3), activation='relu',padding = 'same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(50, (3, 3), activation='relu',padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))              
model.add(Conv2D(100, (3, 3), activation='relu',padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(768, activation='relu'))              
model.add(Dropout(0.2))
model.add(Dense(684, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 32, 32, 20)        560       
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 16, 16, 50)        9050      
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 8, 8, 50)          0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 8, 8, 100)         45100     
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 4, 4, 100)         0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 1600)              0         
__________

In [76]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

filepath="model_chars_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [77]:
train_datagen = ImageDataGenerator(
    rescale=1./255   
)
val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir,
    target_size=(img_width,img_height),
    classes=CLASSES,
    batch_size=batch_size
)
val_generator = val_datagen.flow_from_directory(
    directory=validation_data_dir,
    target_size=(img_width,img_height),
    classes=CLASSES,
    batch_size=batch_size
)   


Found 68400 images belonging to 684 classes.
Found 27360 images belonging to 684 classes.


In [78]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size,
    shuffle = 'batch',
    callbacks = [checkpoint])

model.save('FinalNet_save_1.h5')

Epoch 1/15
2138/2137 [==============================] - 996s 466ms/step - loss: 5.7560 - acc: 0.0273 - val_loss: 3.9712 - val_acc: 0.1485
Epoch 2/15
2138/2137 [==============================] - 434s 203ms/step - loss: 3.1562 - acc: 0.2360 - val_loss: 1.9973 - val_acc: 0.4758
Epoch 3/15
2138/2137 [==============================] - 215s 101ms/step - loss: 1.8540 - acc: 0.4759 - val_loss: 1.2235 - val_acc: 0.6554
Epoch 4/15
2138/2137 [==============================] - 224s 105ms/step - loss: 1.2215 - acc: 0.6328 - val_loss: 0.8387 - val_acc: 0.7541
Epoch 5/15
2138/2137 [==============================] - 239s 112ms/step - loss: 0.8763 - acc: 0.7268 - val_loss: 0.6300 - val_acc: 0.8139
Epoch 6/15
2138/2137 [==============================] - 234s 109ms/step - loss: 0.6580 - acc: 0.7919 - val_loss: 0.4919 - val_acc: 0.8526
Epoch 7/15
2138/2137 [==============================] - 221s 103ms/step - loss: 0.5134 - acc: 0.8345 - val_loss: 0.4302 - val_acc: 0.8682
Epoch 8/15
2138/2137 [============